In [1]:
import numpy as np
from ROOT import TEllipse, TCanvas, TGraph, TLegend

class CovarianceMatrix2DContourPlot(object):
    """

    TODO: extend the class to plot more than one covariance_matrix!
    """

    def __init__(self, mean, covariance_matrix, args = dict(sigma = 1)):
        """ """
        self.mean = mean
        self.covariance_matrix = covariance_matrix
        self.args = args

        if len(self.args) != 1:
            print "args length different than 1"
            return

        if 'sigma' in self.args:
            self.sigma = self.args['sigma']
            self.k = self.sigma
        elif 'probability' in self.args:
            self.probability = self.args['probability']
            self.k = np.sqrt(-2 * np.log(1 - self.probability))
        else:
            print "neither 'sigma' nor 'probability' in args"
            return

        self.secondDerivativeMatrix = np.linalg.inv(self.covarianceMatrix)

        self.eigenvals, self.eigenvecs = np.linalg.eig(self.secondDerivativeMatrix)
        self.lambd = np.sqrt(self.eigenvals)
        self.angle = np.rad2deg(np.arctan(self.eigenvecs[1,0]/self.eigenvecs[0,0]))

        self.ellipse = TEllipse(self.mean[0], self.mean[1], self.k/self.lambd[0], self.k/self.lambd[1], 0., 360., self.angle)
        self.ellipse.SetFillStyle(0)
        self.ellipse.SetLineColor(2)
        self.ellipse.SetLineWidth(4)

        self.scale = max(self.k/self.lambd[0], self.k/self.lambd[1])

Welcome to ROOTaaS 6.06/00


In [2]:
def CombineCovarianceMatrices(covariance_matrix_list):
    """Combine two covariance matrices.

    The arguments must be numpy 2-D array.
    """
    inverse_matrices = []
    for covariance_matrix in covariance_matrix_list:
        inverse_matrices.append( np.linalg.inv(covariance_matrix) )

    alpha = sum(inverse_matrices)
    inverse_alpha = np.linalg.inv(alpha)
    return inverse_alpha

def RotateCovarianceMatrix(covariance_matrix, rotation_matrix):
    rotation_matrix_tr = np.transpose(rotation_matrix)
    second_der_matrix = np.linalg.inv(covariance_matrix)
    second_der_matrix_rotated = rotation_matrix_tr * second_der_matrix * rotation_matrix
    covariance_matrix_rotated = np.linalg.inv(second_der_matrix_rotated)
    return covariance_matrix_rotated

In [3]:
sin2_theta_w = 0.2315
cos_theta_w = np.sqrt(1.-sin2_theta_w)

print "sin cos =", sin2_theta_w, cos_theta_w

Az = (0.5/cos_theta_w)*(1.-(8./3.)*sin2_theta_w)
Bz = (0.5/cos_theta_w)
print "Az, Bz =", Az, Bz

sin cos = 0.2315 0.876641317758
Az, Bz = 0.218257261502 0.570358697305


In [4]:
cova_mu = np.matrix([[  5.64825023e-06,   1.10998336e-05], [  1.10998336e-05,   4.25637322e-04]])
cova_el = np.matrix([[  7.67813611e-06,   1.14758001e-05], [  1.14758001e-05,   5.56975008e-04]])

rotation_matrix = np.matrix([[ (Az+Bz)/2,  (Az-Bz)/2],[  (Az+Bz)/2, (-1)*(Az-Bz)/2]])

In [6]:
total_covariance_matrix = CombineCovarianceMatrices([cova_mu, cova_el])
total_covariance_matrix_rotated = RotateCovarianceMatrix(total_covariance_matrix, rotation_matrix)